In [2]:
import numpy as np

In [1]:
problem_set = [
    [["0", "2"], ["1", "0"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "3"]],
    [["0", "0"], ["1", "1"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "1"], ["1", "3"], ["3", "2"], ["3", "3"]],
    [["0", "2"], ["1", "0"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "1"], ["3", "3"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "3"], ["1", "2"], ["2", "2"], ["2", "3"], ["3", "0"]],
    [["0", "3"], ["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "3"], ["2", "1"], ["2", "3"], ["3", "0"]],
    [["0", "1"], ["0", "3"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "2"]],
    [["0", "0"], ["1", "3"], ["2", "0"], ["2", "1"], ["2", "3"], ["3", "1"]],
    [["0", "1"], ["0", "2"], ["1", "0"], ["1", "2"], ["2", "2"], ["2", "3"]],
    [["0", "3"], ["1", "0"], ["1", "3"], ["2", "1"], ["2", "2"], ["3", "0"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "3"], ["3", "0"], ["3", "1"]],
    [["0", "1"], ["1", "0"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "1"]],
]


In [66]:
def row_to_dict(row):

    row_dict = {}
    for i, item in enumerate(row):

        if len(row) == 2:
            bin_str = f"{i:01b}"
        if len(row) == 4:
            bin_str = f"{i:02b}"
        if len(row) == 8:
            bin_str = f"{i:02b}"
        if len(row) == 16:
            bin_str = f"{i:04b}"

        row_dict[bin_str] = item

    return row_dict


In [69]:


def print_and_split(cluster_dict, bit_string="", branch_order=[3,2,1,0]):

    score_array = [10, 60, 180, 300, 420]

    # Check to see if there is only one value in the dictionary.values()
    if len(set(cluster_dict.values())) == 1:
        unique_value = set(cluster_dict.values()).pop()
        #print(f"We have a leaf with value {unique_value} with bit_string {bit_string}")
        return score_array[len(bit_string)]
    
    zero_dict = {}
    one_dict = {}
    current_bit = branch_order[0]
    
    for k,v in cluster_dict.items():
        if k[current_bit] == "0":
            zero_dict[k] = v
        else:
            one_dict[k] = v

    score1 = print_and_split(zero_dict, bit_string + "0", branch_order[1:])
    score2 = print_and_split(one_dict, bit_string + "1", branch_order[1:])

    return score1 + score2

perm = [3,1,0,2]
score = print_and_split(row_to_dict(test), branch_order=perm)
print(f"The score is {score} with permutation {perm}; worst score is {420*16}")

The score is 5100 with permutation [3, 1, 0, 2]; worst score is 6720


In [73]:
ps = np.array(problem_set, int)
cluster_horz = ps.T[0]
cluster_vert = ps.T[1]

test = cluster_horz[4]

row_to_dict(test)



for i, row in enumerate(cluster_horz):
    best_score = 9999
    best_perm = None

    for p in permutations([0,1,2,3]):
        score = print_and_split(row_to_dict(row), branch_order=p)
        if score < best_score:
            best_score = score
            best_perm = p

    score = print_and_split(row_to_dict(test), branch_order=best_perm)
    print(f"The best score is for horizontal row {i} is {score} with permutation {best_perm}; worst score is {420*16}")

for i, row in enumerate(cluster_vert):
    best_score = 9999
    best_perm = None

    for p in permutations([0,1,2,3]):
        score = print_and_split(row_to_dict(row), branch_order=p)
        if score < best_score:
            best_score = score
            best_perm = p

    score = print_and_split(row_to_dict(test), branch_order=best_perm)
    print(f"The best score is for vertical row {i} is {score} with permutation {best_perm}; worst score is {420*16}")

The best score is for horizontal row 0 is 2520 with permutation (0, 1, 2, 3); worst score is 6720
The best score is for horizontal row 1 is 4680 with permutation (0, 1, 3, 2); worst score is 6720
The best score is for horizontal row 2 is 5100 with permutation (3, 0, 1, 2); worst score is 6720
The best score is for horizontal row 3 is 2520 with permutation (0, 1, 2, 3); worst score is 6720
The best score is for horizontal row 4 is 2520 with permutation (0, 1, 2, 3); worst score is 6720
The best score is for horizontal row 5 is 2520 with permutation (0, 1, 2, 3); worst score is 6720
The best score is for vertical row 0 is 5100 with permutation (1, 3, 0, 2); worst score is 6720
The best score is for vertical row 1 is 2520 with permutation (0, 1, 2, 3); worst score is 6720
The best score is for vertical row 2 is 2940 with permutation (1, 2, 0, 3); worst score is 6720
The best score is for vertical row 3 is 4680 with permutation (0, 2, 3, 1); worst score is 6720
The best score is for vertic